In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from json import load
import tifffile
from tqdm import tqdm
from skimage.draw import polygon
from matplotlib import rcParams
from glasbey import create_palette

palette = {
    "green": "#558150",
    "beige": "#F1E2C3",
    "brown": "#A7785A",
    "pink": "#F0D6C2",
    "black": "#0E0E0E",
}

rcParams['font.family'] = 'sans-serif'
rcParams['figure.facecolor'] = "#FFFFFF00"
rcParams['axes.facecolor'] = "#FFFFFF00"
rcParams['legend.framealpha'] = 0.2
rcParams['axes.edgecolor'] = palette["black"]
rcParams['axes.labelcolor'] = palette["black"]
rcParams['xtick.color'] = palette["black"]
rcParams['ytick.color'] = palette["black"]
rcParams['text.color'] = palette["black"]
rcParams['axes.titlecolor'] = palette["black"]

In [21]:
source_path = Path(r"D:\Tracking\NucleiTracking\data\interim\lightsheet\2025_01_31\recon")
spots_df = pd.read_csv(source_path / r"out\best_spots.csv")
save_path = source_path / "wave_animations_2"
save_path.mkdir(exist_ok=True)

In [30]:
import napari
from sklearn.neighbors import NearestNeighbors
import numpy as np


def load_frame_data(df, frame, neighbors=5):
    frame_subset = df[df["FRAME"] == frame].copy()
    values = np.array([frame - mid for mid in frame_subset["cycle_start"]])
    values[values > 12] = -1
    values[(values >= 0) & (values <= 1)] = 0
    values[(values <= 12) & (values > 1)] = 1
    spatial = frame_subset[["POSITION_X", "POSITION_Y", "POSITION_Z"]].values
    
    knn = NearestNeighbors(n_neighbors=neighbors)
    knn.fit(spatial, values)
    fit_values = values[knn.kneighbors(spatial, return_distance=False)]
    fit_values = np.array(np.median(fit_values, axis=1), dtype=int)
    frame_subset["true_value"] = values
    frame_subset["fit_value"] = fit_values
    return frame_subset
    
spots_df["cycle_start"] = spots_df["tracklet_id"].map(spots_df.groupby("tracklet_id")["FRAME"].min())

start_frame = 198
end_frame = 212
rotation_per_image = 0.1
rotation_per_frame = np.pi

total_rotation = 0
frame_index = 0
frame_data = load_frame_data(spots_df, start_frame)
s = 10
border_color = "black"


print(frame_data["true_value"], frame_data["fit_value"])

viewer = napari.Viewer(ndisplay=3)
# viewer._canvas_size = (900, 900)
viewer.theme = 'dark'
i = 0

loader = tqdm(total=(end_frame - start_frame) * rotation_per_frame // rotation_per_image)

while frame_index < end_frame - start_frame:
    i += 1
    
    loader.update(1)
    
    if total_rotation // rotation_per_frame > frame_index:
        frame_index += 1
        frame_data = load_frame_data(spots_df, start_frame + frame_index)
    
    cmap = {-1: "#2E4B36", 0: "#FF5638", 1: "#FF7530"}
    color = [cmap[v] for v in frame_data["fit_value"]]
    pts = frame_data[["POSITION_X", "POSITION_Y", "POSITION_Z"]].values
    pts = pts - pts.mean(axis=0)
    
    rot_matrix = np.array([[np.cos(total_rotation), 0, -np.sin(total_rotation)],
                           [0, 1, 0],
                           [np.sin(total_rotation), 0, np.cos(total_rotation)]])
    pts = pts @ rot_matrix
    
    l = viewer.add_points(pts, name=f"frame_{i}", border_color=border_color, border_width=0.04,
                              face_color=color, size=s, translate=[100, 0, 0], shading='none', antialiasing=0.5)
    l.refresh()
    viewer.screenshot(str(save_path / f"frame_{i}.png"), scale=1, flash=False)
    viewer.layers.remove(l.name)
    del l
    
    
    if i % 30 == 0:
        viewer.close()
        viewer = napari.Viewer(ndisplay=3)
        # viewer._canvas_size = (900, 900)
        viewer.theme = 'dark'
        
    else:
        total_rotation += rotation_per_image
    

266652   -1.0
266653   -1.0
266654   -1.0
266655   -1.0
266656   -1.0
         ... 
269184   -1.0
269185   -1.0
269186   -1.0
269187   -1.0
269188   -1.0
Name: true_value, Length: 2537, dtype: float64 266652   -1
266653   -1
266654   -1
266655   -1
266656   -1
         ..
269184   -1
269185   -1
269186   -1
269187   -1
269188   -1
Name: fit_value, Length: 2537, dtype: int32



50it [01:28,  1.77s/it]0 [00:00<?, ?it/s]

  0%|          | 2/439.0 [00:00<02:03,  3.55it/s]
  1%|          | 3/439.0 [00:01<02:43,  2.67it/s]
  1%|          | 4/439.0 [00:01<03:02,  2.38it/s]
  1%|          | 5/439.0 [00:02<03:15,  2.22it/s]
  1%|▏         | 6/439.0 [00:02<03:23,  2.13it/s]
  2%|▏         | 7/439.0 [00:03<03:30,  2.06it/s]
  2%|▏         | 8/439.0 [00:03<03:37,  1.98it/s]
  2%|▏         | 9/439.0 [00:04<03:42,  1.94it/s]
  2%|▏         | 10/439.0 [00:04<03:43,  1.92it/s]
  3%|▎         | 11/439.0 [00:05<03:44,  1.91it/s]
  3%|▎         | 12/439.0 [00:05<03:44,  1.90it/s]
  3%|▎         | 13/439.0 [00:06<03:46,  1.88it/s]
  3%|▎         | 14/439.0 [00:06<03:47,  1.87it/s]
  3%|▎         | 15/439.0 [00:07<04:05,  1.72it/s]
  4%|▎         | 16/439.0 [00:08<04:02,  1.75it/s]
  4%|▍         | 17/439.0 [00:08<03:59,  1.76it/s]
  4%|▍         | 18/439.0 [00:09<03:56,  1.78it/s]
  4%|▍         | 19/439.0 [00:09<03:56,  1.78it/s]
  5%|▍         | 20/439.0 [00:10<03:55,  1.78i